# <!-- TITLE --> [GRISLI temperature] 
## Regression with a Dense Neural Network (DNN)  


- AUTHOR : catritz 
- date 18 janvier2021
- reprise 13 février 2021
- reprise (pour EGU) le 21 Avril 2021

## version with time dependent surface temperature as input
- on **difference with surface temperature**
-  using other predictors such as **varTb**
- reading data from xarray


## Objectives :
 - Predicts **temperature profile** from a set of parameters
 - with a **dense neural network**  


### Database : 
- generated with read-GRISLI-xarray.ipynb
- read-GRISLI-xarray-time-dep.ipynb

- data in ./data/Grisli_4_DNN_AN40C008.nc and runs 8-11 
- and in ./data/Grisli_4_DNN_time_dep_AN40C008.nc runs 8-11

- variables

    * H: ice thickness (m)
    * ghf (in mw/m2)
    * Ts:surface temperature (°C)
    * acc: accumulation rate (m/yr)
    * base: basal temperature (difference from the melting point)    
    
- time dependent surface temperature
    * Tann over the last 150 k, take only the last 25 k
    
 
 **The vector of temperature profile**
 - T_00,T_01,T_02,T_03,T_04,T_05,T_06,T_07,T_08,T_09,T_10,T_11,T_12,T_13,T_14,T_15,T_16,T_17,T_18,T_19,T_20
 
 ***


## What we're going to do :

 
### 1. define title and parameters of the run 
- 1.1 define all parameters
- 1.2 store parameters in a dictionnary <span style='color:blue '> dictionnary **docDNN** </span>   
     
### 2. Import the data to build the model 
- Retrieve data in netcdf as a xarray dataset  
 > pd_onefile = nc2pd(file_in) 
- loop on several netcdf
- final <span style='color:blue '> dataframe **df_tot** </span>

### 3. Prepare the data

According to what has been defined in the parameter list

- 3.1 remove floating part, thickness too small and eventually other caracteristics
> data2 = select_some(df_tot,docDNN)
- 3.2 to work in difference with surface temperature
> data2 = diff_surf(data2,docDNN )
- 3.3 split the data between train and test
<span style='color:blue '> **data_train and data_test** </span>
- 3.4 split x (predictor) and y (target) : <span style='color:blue '> dataframes **x_train, x_test, y_train, y_test** </span>
- 3.5 - Data normalization
    * 3.5.1 normalization itself ->  <span style='color:blue '> dataframes **x_trainnorm, x_testnorm** </span>
    - to use the normalisation
    > newx = get_norm(x,dic)    
    > where x is a dataframe
    
    * 3.5.2 Save the characteristics of the run in .json file including normalization: mean_list, std_list
- 3.6 back to <span style='color:blue '> numpy arrays : **x_train2, x_test2, y_train2 , y_test2** </span> (one keep dimension values in numpy arrays: x_traintrue, x_testrue)
 
## 4. Build a model

## 5. - Train the model
- 5.1  Get the model and show a display of the model
- 5.2  Define model check points 
- 5.3  Train 

## 6.  Evaluate
- 6.1 Model evaluation
- 6.2 training history 
- 6.3 Retrieve the best model obtained

## 7. Predict

***



 
 
## lectures
- geometrie du modèle
https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
- loss
 https://towardsdatascience.com/advanced-keras-constructing-complex-custom-losses-and-metrics-c07ca130a618
 
## to import my own modules
https://python.sdv.univ-paris-diderot.fr/14_creation_modules/

***

## Import Modules

In [ ]:
import tensorflow as tf
from tensorflow import keras
#from keras.layers import Dropout
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm

import pandas as pd
import os,sys
import xarray as xr
import math
from datetime import date
import json

import seaborn as sns

sys.path.append('..')
import fidle.pwk as ooo

ooo.init()
os.makedirs('./run/',   mode=0o750, exist_ok=True)

## Utilitary routines

### pretty print of a dictionnary

In [ ]:
def pretty(d, indent = 1):   # pretty print d'un dictionnaire
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))
        print()

### subroutine nc2pdtd : read netcdf files and construct a dataframe with time dependent Tann

In [ ]:
def nc2pdtd(file_in):     ### step 2. read a netcdf file from GRISLI obtained with : read-GRISLI-xarray.ipynb
                           ### time dependent version

    DD = xr.open_dataset(file_in)  #

    
### 2D variables at present time

    Dss = DD.sel(z=0).drop("T3D")   # drop la variable 3D du xarray dataset

    pd_2D = Dss.drop("z").to_dataframe()  # drop z and convert to dataframe
    

 ### time dependent surface temperature in x_array D_td

    # derive tjan-fev-2021-hdf5-jsonhe name of the time dependent file from file_in
    numtime = 25   # numer of time slices taken into account
    s='DNN_'
    pos = file_in.rfind(s)
    file_td = file_in[0:pos+len(s)]+'time_dep_'+ file_in[pos+len(s):]

    # read the dataset
    D_Td = xr.open_dataset(file_td)
    print(D_Td.dims)

    # keep only Tann and select a time subset
#    Tann = D_Td.Tann.sel(time=slice(-numtime*1000,0)) # keep only Tann on numtime kyears
    timelist = [-x*1000 for x in range(numtime+1)] 

    Tann = D_Td.Tann.sel(time = timelist) # keep only Tann on numtime kyears

    # Convert in a dataframe in which each time slice will be a column
    Time_cols = ["Tann_"+ str(x).zfill(2)+"kyr" for x in range(numtime+1)]  # list columns
    #print (Time_cols)

    dTann = Tann.to_dataframe()
    dTann = dTann.unstack(level='time').swaplevel().sort_index() # move the time in columns
                                                                 # at this index level "time" still 
                                                                 # appears as a sign of multiindex
    
    dTann.columns = Time_cols                                   # remove the index level "time"
    
    
    pd_2Dtd = pd.merge(pd_2D,dTann,how='left',on=['x','y'])   
    
### 3D variablescharacteristics

    dft = DD.T3D.to_dataframe()

    dT3D = dft.unstack(level='z').swaplevel().sort_index()

    # change the column names to get rid of level 'z' in the dataframe
    dT3D.columns = (['T_00','T_01','T_02','T_03','T_04','T_05',
                     'T_06','T_07','T_08','T_09','T_10',
                    'T_11','T_12','T_13','T_14','T_15',
                     'T_16','T_17','T_18','T_19','T_20'])
    
       

    pd_onefile = pd.merge(pd_2Dtd,dT3D,how='left',on=['x','y'])
    pd_onefile.reset_index(level=[0,1], inplace=True)      # transform indexes x and y to columns
    
    

# extract the name of the GRISLI run and fill a column with it to enable sorting a specific case later.
    posdeb = file_in.rfind('/')
    posend = file_in.rfind('.nc')
    subfile = file_in[posdeb+1:posend]    
    pd_onefile['GRISLI_run'] = subfile    
 
    
    return pd_onefile


### subroutine select_some : select points according to criteria in the dictionnary dic

In [ ]:
def select_some(df_tot,dic):   # select points according to criteria in dictionnary. section 3.1
# return the dataframe data2    
   
    Hmin = dic['select_data']['thickmin'] 
    base_type = dic['select_data']['base']
    mk = dic['select_data']['mk']
    DTb = dic['select_data']['varTb'] 

    datagrounded = df_tot[df_tot.Mk < 0.5] 


    if DTb == 0:
        datagrounded = datagrounded[datagrounded.VarTb == 0] 


    data2 =  datagrounded[datagrounded.H > Hmin] 

    if base_type == 'cold':
        data2 =  data2[data2.Tb < 0] 

    elif base_type == 'wet':
        data2 =  data2[data2.Tb == 0] 

    else: 
        data2 =  data2

    return data2

### subroutine to make the difference with the surface 

In [ ]:
def diff_surf(df_in, dic):  # make the difference with surface temperature step 3.2
    list_y = dic['y_columns']['list_y']
    
    df2 = df_in.copy(deep=True)  # if not, modify the original dataframe(warning)
    df2[list_y] = df_in[list_y].sub(df_in.Ts,axis=0)
    df_out = df2
    
    return df_out

###  subroutine diff_model_data(ydiff, df_in)
- Make differences between predicted and target and merge with the initial dataframes 
- return the concatenated dataframe


In [ ]:
#  Make differences between predicted and target and merge with the initial dataframes step 7.4
# ydiff is obstained by :
# ypred = model.predict (x)  where x is normalized
# diff = ypred - y           where y is the  initial data once sorted and eventually in difference with surface temperature

def diff_model_data(ydiff, df_in):

    nbrow = ydiff.shape[1]   # number of elements in the vertical
    
    columns_diff = ['mse_row','sigma_row','mae_row','me_row','dT_01', 'dT_02', 'dT_03', 'dT_04', 'dT_05',
                'dT_06', 'dT_07', 'dT_08', 'dT_09', 'dT_10', 'dT_11', 'dT_12', 'dT_13', 'dT_14',
                'dT_15', 'dT_16', 'dT_17', 'dT_18', 'dT_19', 'dT_20']             # to name the diff columns
    
    # the bloc below calculate mse_row, mae_row, me_row and stack them with the ydiff 2D values
    # each row represent a physical vertical column
    
    yd2    = ydiff**2
    ydabs = np.abs(ydiff)
    mse_row   = yd2.sum(axis=1)/nbrow        # mse row
    sigma_row = np.sqrt(mse_row)             # sigma_row
    mae_row   = ydabs.sum(axis=1)/nbrow      # mae row
    me_row    = ydiff.sum(axis=1)/nbrow      # difference with sign
    
    all_diff = np.column_stack((mse_row,sigma_row,mae_row,me_row,ydiff))
    
    ### Merge differences with the initial dataframe df_in
    ddiff = pd.DataFrame(all_diff, columns=columns_diff,index=df_in.index)
    df_out = pd.concat([df_in,ddiff],axis=1,ignore_index=False)

    return df_out


### subroutine get_norm : normalize df according to parameters and norms from dictionnary

In [ ]:
def get_norm(df,dic):    # calculate norms from df according to parameters
                         # in dic and return x_norm the dataframe with 
                         # the normalized columns of list_pred
    
    normalize = dic['normalization']['method']
    list_pred = dic['x_columns']['list_pred']   # to reconstruct the mean dataframe


    if normalize == 'mean':  # mean std normalisation

        # mean_list and std_list are list
        mean_list = dic['normalization']['xmean']
        std_list  = dic['normalization']['xstd']

        # to convert list to dataframe
        mean = pd.DataFrame(mean_list,index=list_pred)
        std = pd.DataFrame(std_list, index=list_pred)
        mean = mean.squeeze()
        std = std.squeeze()       

        #keep only only the columns 'list_pred' in df
        x_norm = (pd.Dataframe(df, columns=list_pred) - mean ) /std

 

    elif normalize == 'meanlog': # replace with the log10 columns from list_meanlog
                                 # limited to 1.e-7 to avoid log10 warning
                                 # then apply usual mean and std on all columns 

        list_meanlog = dic['normalization']['columns']
        mean_list = dic['normalization']['xmean']
        std_list  = dic['normalization']['xstd']

        #keep only only the columns 'list_pred' in df
        xx = pd.DataFrame(df, columns=list_pred)

        # take the log x_train
        x_log = xx.copy(deep=True)       # to avoid changing xx
        x_tolog   = x_log[list_meanlog]  # keep only the columns that must be logDNN_with_varT
        x_tolog   = np.maximum(x_tolog, 1.e-7)  # to avoid 0
        x_logged  = np.log10(x_tolog)

        
        x_log[list_meanlog] = x_logged
 
        # to convert list to dataframe
#       pivot constructs a dataframe with columns    
#        mean = pd.DataFrame(mean_list).pivot_table(columns=list_pred)
#        std  = pd.DataFrame(std_list).pivot_table(columns=list_pred)
        mean = pd.DataFrame(mean_list,index=list_pred)
        std = pd.DataFrame(std_list, index=list_pred)
        mean = mean.squeeze()
        std = std.squeeze()    
 
        x_norm = (x_log - mean) / std  
 
    elif normalize == 'minmax': # min-max normalisation
        
        minx_list = dic['normalization']['minx']
        maxx_list = dic['normalization']['maxx']
      
        minx = pd.DataFrame(minx_list, index=list_pred)
        maxx = pd.DataFrame(maxx_list, index=list_pred)
        minx = minx.squeeze()
        maxx = maxx.squeeze()
        
        #keep only only the columns 'list_pred' in df
        xx = pd.Dataframe(df, columns=list_pred)
        
        x_norm = (xx - minx) / (maxx - minx)
        
    return x_norm


## 1. define title and parameters of the run


- Title and list of GRISLI files to use 


- conditions on the nodes (thickness min, type of base)  


- list of predictors   
    *  possibilities: 'x', 'y', 'S', 'H', 'B', 'Mk', 'Ts', 'Tb', 'Bm', 'ghf', 'slope', 'Us',
       'VarTb', 'Bmin', 'Bmax', 'Bminmax', 'Bmedian', 'Bmean', 'Bstd', 
       'crx','cry','crxy' 
    * types of regularisation. possibilities mean, meanlog or minmax
    
    
- list of target   
    * the vertical temperature profile with parameters
    * with some parameters
    
    
- parameters will be stored in a dictionnary docDNN later writen in a json file      

### 1.1 define all parameters

## loop on the models

In [ ]:
#dirmodel = 'apr-2021-hdf5-json/'
dirmodel = ''
subtitle = 'april2021_test_'
postitle = '_all_meanlog'

#ndeb = 7
#nend = 17
ndeb = 30
nend = 40

for i in range(ndeb,nend):
    # retrieve the title
    numstring = "{:02n}".format(i)
    title = dirmodel + subtitle + numstring + postitle  
    print (title)  
    
    # data files from GRISLI
    file08 = './data/Grisli_4_DNN_AN40C008.nc'
    file09 = './data/Grisli_4_DNN_AN40C009.nc'
    file10 = './data/Grisli_4_DNN_AN40C010.nc'
    file11 = './data/Grisli_4_DNN_AN40C011.nc'

    ncfile_list = [file08,file09,file10,file11]
    
    ### apply conditions on nodes 
    mk = 'grounded'    
    Hmin = 1000
    base_type = 'all' # 'all' # 'cold', wet' or 'all' 
    DTb = 0  # takes only point that have neighbours with the same thermal condition

    ### characteristics that are used in predictor

    # possibilities : 'x', 'y', 'S', 'H', 'B', 'Mk', 'Ts', 'Tb', 'Bm', 'ghf', 'slope', 'Us',
    #       'VarTb', 'Bmin', 'Bmax', 'Bminmax', 'Bmedian', 'Bmean', 'Bstd', 
    #       'crx','cry','crxy' 

    list_pred = ['H','Ts', 'Bm', 'ghf', 'slope', 'Us']  
    list_Tann =['Tann_01kyr', 'Tann_02kyr', 'Tann_03kyr', 'Tann_04kyr', 'Tann_05kyr',
                'Tann_06kyr', 'Tann_07kyr', 'Tann_08kyr', 'Tann_09kyr', 'Tann_10kyr', 
                'Tann_11kyr', 'Tann_12kyr', 'Tann_13kyr', 'Tann_14kyr', 'Tann_15kyr',
                'Tann_16kyr', 'Tann_17kyr', 'Tann_18kyr', 'Tann_19kyr', 'Tann_20kyr',
                'Tann_21kyr', 'Tann_22kyr', 'Tann_23kyr', 'Tann_24kyr']

    list_pred = list_pred + list_Tann
    num_x = len(list_pred)   # number of predictors

    # type of regularisation
    utaub = 'none'
    normalize = 'meanlog' #  possibilities mean, meanlog or minmax
    list_meanlog = ['Bm','slope','Us'] # characteristics for which meanlog is used


    # target : output vector
    list_target = ['T_00', 'T_01', 'T_02', 'T_03', 'T_04', 'T_05', 'T_06','T_07', 'T_08', 'T_09', 
                   'T_10', 'T_11', 'T_12', 'T_13', 'T_14', 'T_15','T_16', 'T_17', 'T_18', 'T_19', 'T_20']

    itop = 1  # 1-> does not try to fit the surface temperature
    ibottom = 20
    nbvect = ibottom - itop + 1   # number of elements in the output vector
    list_y = list_target[itop:ibottom+1]
    # print(list_y)

    # date
    today = date.today()
    today = today.strftime("%d/%m/%Y")
    
    ### 1.2 store parameters in a dictionnary

    #- normalisation dictionnary is done in the docDNN later
    # fill the parameters in the dictionnary. The dictionnary will be writen in a .json file

    # fill the parameters in the dictionnary. The dictionnary will be writen in a .json file
    # each topic is a subdictionnary


    docDNN = {}                     # create a dictionnary docDNN
    docDNN['titles'] = {'main': title,'date': today }           # create and fill a key titles
    docDNN['imported_data'] = {'file_list': ncfile_list}
    docDNN['select_data'] = {'thickmin': Hmin,'base': base_type,'mk': mk,'varTb': DTb }

    docDNN['x_columns'] = {'list_pred': list_pred,'Utaub': utaub}
    docDNN['y_columns'] = {'itop': itop,'ibottom':ibottom,'list_y': list_y}

    docDNN['normalization'] = {'method': normalize}   # the end of normalization is stored later 

    ## 2. Import the data to build the model
    # Read thet netcdf and convert in a dataframe
    # loop on the grisli databases (ncfiles)
    # in the subroutine nc2df (file_in)

    #At the end of this bloc, <span style='color:blue '> **df_tot** is the dataframe containing all the data </span>
    
    # initialize DataFrame before loop
    df_tot = pd.DataFrame()

    # loop on the ncfiles
    for file_in in ncfile_list:
#        print (file_in)
        pd_onefile = nc2pdtd(file_in)
        df_tot = df_tot.append(pd_onefile,ignore_index=True)  
        
    ### columns of the initial database df_tot
    #df_tot.columns
    
    ## 3. Prepare the data
    ### 3.1 remove floating part, thickness too small and eventually other caracteristics

    # consider removing points where VarTb is too high
 
    data2 = select_some(df_tot,docDNN)
    # data2.columns
    #display(data2.describe().style.format("{0:.2f}").set_caption("data2"))
    
    ### 3.2 to work in difference with surface temperature 

    data2 = diff_surf(data2,docDNN )
    # data2.shape

    ### 3.3 Split data
    # At the end of this bloc  data_train and data_test

    data_train = data2.sample(frac=0.7, axis=0)   # axis = 0 for the index
    data_test  = data2.drop(data_train.index)
    
    ### 3.4 split x (predictor) and y (target)

    # keep only some of the input and output columns (according to parameters)
    # predictors in x
    # target in y
    # At the end of this bloc  dataframes x_train, x_test, y_train, y_test
    # num_x is the number of columns in x_train
    x_train = pd.DataFrame(data_train,columns=list_pred)  
    x_test = pd.DataFrame(data_test,columns=list_pred)  

    # nbvect is the number of columns in y_train
    y_train = pd.DataFrame(data_train,columns=list_y)  
    y_test = pd.DataFrame(data_test,columns=list_y)  

    # eventually add Utaub and/or U2 (see DNN-tests-TGrisli notebook)
    #x_train.columns
    
    ### 3.5 - Data normalization
    # display(x_train.describe().style.format("{0:.2f}").set_caption("Before normalization :"))
    
    # normalization 

    if normalize == 'mean':  # mean std normalisation
        mean = x_train.mean()
        std  = x_train.std()
        x_trainnorm = (x_train - mean) / std
        x_testnorm  = (x_test  - mean) / std  # note x_test is normalized with the
                                              # x_train mean and std

        mean_list = mean.values.tolist()      # json only accept lists
        std_list  = std.values.tolist()

        docDNN['normalization']['xmean'] = mean_list
        docDNN['normalization']['xstd'] = std_list


    elif normalize == 'meanlog': # replace with the log10 columns from list_meanlog
                                 # limited to 1.e-7 to avoid log10 warning
                                 # then apply usual mean and std on all columns 

        # take the log x_train
        x_trainlog = x_train.copy(deep=True)
        train_tolog   = x_trainlog[list_meanlog]  # keep only the columns that must be log
        train_tolog   = np.maximum(train_tolog, 1.e-7)  # to avoid 0
        train_logged  = np.log10(train_tolog)

        x_trainlog[list_meanlog] = train_logged

        mean = x_trainlog.mean()
        std  = x_trainlog.std()   


        x_trainnorm = (x_trainlog - mean) / std


        # x_test
        x_testlog = x_test.copy(deep=True)
        test_tolog   = x_testlog[list_meanlog]   # keep only velocity and slope
        test_tolog   = np.maximum(test_tolog, 1.e-7)  # to avoid 0
        test_logged  = np.log10(test_tolog)    
        x_testlog[list_meanlog] = test_logged


        x_testnorm  = (x_testlog  - mean) / std  # note x_test is normalized with 
                                                 # the x_train mean and std

    #   save the normalisation values in a list 
        mean_list = mean.values.tolist()
        std_list  = std.values.tolist()

        docDNN['normalization']['columns'] = list_meanlog
        docDNN['normalization']['xmean'] = mean_list
        docDNN['normalization']['xstd'] = std_list



    elif normalize == 'minmax': # min-max normalisation
        minx = x_train.min()
        maxx = x_train.max()
        x_trainnorm = (x_train - minx) / (maxx - minx)
        x_testnorm = (x_test - minx)/ (maxx - minx)

        minx_list = minx.values.tolist()
        maxx_list = maxx.values.tolist()


        docDNN['normalization']['minx'] = minx_list
        docDNN['normalization']['maxx'] = maxx_list 
        
    #### 3.5.2 Save the characteristics of the run including normalization: mean_list, std_list
    
    with open(title + '.json', 'w') as outfile:             # parameters of input preprocesing in json 
        json.dump(docDNN, outfile,indent=4)
        
    #display(x_trainnorm.describe().style.format("{0:.2f}").set_caption("after normalization :"))

    ## 3.6 back to numpy arrays
    x_train2 = np.array(x_trainnorm)
    x_test2  = np.array(x_testnorm)

    y_train2 = np.array(y_train)
    y_test2  = np.array(y_test)

    x_traintrue = np.array(x_train) 
    x_testtrue  = np.array(x_test)
    
    ## 4. Build a model
    def get_model_v1(shape):
        nodes = 256
        #activ = 'sigmoid'
        activ = 'relu'   # standard
        #activ = 'tanh'
        #activ = 'selu'
        model = keras.models.Sequential()
        model.add(keras.layers.Input(shape, name="InputLayer"))
        model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n1'))
        model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n2'))
        model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n3'))
        model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n4'))
        model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n5'))
        model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n6'))
        model.add(keras.layers.Dense(nbvect, name='Output'))  
                                        # nbvect = number of elements of target vector

        model.compile(optimizer = 'rmsprop',
                      loss      = 'mse',                  # mse, mean quadratic error 
                  metrics   = ['mae', 'mse'] )        # mae =  mean absolute error
        return model
 
    ### 5.1  Get the model and show a display of the model
    model=get_model_v1( (num_x,) )  # num_x number of predictors = shape of x_train or x_test. 

    ### show a display of the model 

    model.summary()
    #img=keras.utils.plot_model( model, to_file='./run/model.png', 
    #                           show_shapes=True, show_layer_names=True, dpi=96)
    #display(img)
    
    

    ### 5.2 Define model check points 
    # mcp : model check point to save the best model



    filepath=title+"-weights-impr-{epoch:02d}-{val_mae:.2f}.hdf5"
    # print(filepath)
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, 
                                                 monitor='val_mae', 
                                                 verbose=1, 
                                                 save_best_only=True, 
                                                 mode='min')
    callbacks_list = [checkpoint]


    ### 5.3 - Train the model

    history = model.fit(x_train2,
                        y_train2,
                        epochs          = 100,
                        batch_size      = 20,
                        verbose         = 1,
                        validation_data = (x_test2, y_test2),
                        callbacks       = callbacks_list)
    ## 6.  Evaluate

    ### 6.1 - Model evaluation
    print ('evaluation ', title)
    score = model.evaluate(x_test2, y_test2, verbose=0)

    print('x_test / loss      : {:5.4f}'.format(score[0]))
    print('x_test / mae       : {:5.4f}'.format(score[1]))
    print('x_test / mse       : {:5.4f}'.format(score[2]))

    ### 6.2 - Training history

    df=pd.DataFrame(data=history.history)
    df.describe()

    # print("min( val_mae ) : {:.4f}".format( min(history.history["val_mae"]) ) )
    %matplotlib inline
    ooo.plot_history(history, plot={'MSE' :['val_mse', 'mse'],
                                'MAE' :['val_mae', 'mae']})
    
    ### 6.3 Retrieve the best model obtained

    # it is the last written with filepath : bestmodel
    # remove the other models saved from the same training
    dirs = os.listdir()   # list the current dir

    # sorted(dirs)          # liste dirs sorted 

    sub = title+'-weight'                                # select the model names, order and take the last one
    list_models = sorted([s for s in dirs if sub in s])  # if sub in s ->  if sub is a substring of s
#    print(dirs,sub)
#    print (list_models)
    bestmodel = list_models[-1]                          # take the last one which is the best

    remove_files=list_models[0:len(list_models)-1]
    for s in remove_files:
        os.remove(s)
    print('bestmodel=' , bestmodel)
